In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
import re
import math
pd.set_option("display.max_rows",101)
pd.set_option("display.max_columns",101)

# Parsing the variants.csv file

In [2]:
mgdata = pd.read_csv("variants_ALL4_to_MG1655v3.csv",dtype = {'IS_HET': str, 'QA': str} )
mgdata = mgdata.loc[:,['UID','POSITION','EXPERIMENT_SAMPLE_LABEL','REF','ALT','DP','AF','INFO_EFF_GENE']]
mgdata = mgdata.rename(columns={'EXPERIMENT_SAMPLE_LABEL':'lineage','POSITION':'position','INFO_EFF_GENE':'gene'})
mgdata = mgdata[mgdata.lineage.notnull()]
c3data = pd.read_csv("variants_ALL4_to_C321.csv",dtype = {'IS_HET': str, 'QA': str} )
c3data = c3data.loc[:,['UID','POSITION','EXPERIMENT_SAMPLE_LABEL','REF','ALT','DP','AF','INFO_EFF_GENE']]
c3data = c3data.rename(columns={'EXPERIMENT_SAMPLE_LABEL':'lineage','POSITION':'position','INFO_EFF_GENE':'gene'})
c3data = c3data[c3data.lineage.notnull()]

# reassign P populations because of mis-labeling of samples somehow
pDic = {'P-1-1':'P-1-1','P-12-2':'P-1-2',
        'P-1-2':'P-2-1','P-13-1':'P-2-2',
        'P-2-1':'P-3-1','P-11-2':'P-3-2',
        'P-2-2':'P-4-1','P-12-1':'P-4-2',
        'P-3-1':'P-5-1','P-6-2':'P-5-2',
        'P-3-2':'P-6-1','P-7-1':'P-6-2',
        'P-4-1':'P-7-1','P-7-2':'P-7-2',
        'P-4-2':'P-8-1','P-8-1':'P-8-2',
        'P-5-1':'P-9-1','P-8-2':'P-9-2',
        'P-5-2':'P-10-1','P-9-1':'P-10-2',
        'P-6-1':'P-11-1','P-9-2':'P-11-2',
        'P-10-1':'P-12-1','P-13-2':'P-12-2',
        'P-10-2':'P-13-1','P-14-1':'P-13-2',
        'P-11-1':'P-14-1','P-14-2':'P-14-2'}

c3data['lineage'] = c3data['lineage'].apply(lambda x: pDic[x] if x in pDic.keys() else x)
mgdata['lineage'] = mgdata['lineage'].apply(lambda x: pDic[x] if x in pDic.keys() else x)

c_count = mgdata[mgdata.lineage.str[0] == 'C'].lineage.nunique()
e_count = mgdata[mgdata.lineage.str[0] == 'E'].lineage.nunique()
f_count = mgdata[mgdata.lineage.str[0] == 'F'].lineage.nunique()
p_count = mgdata[mgdata.lineage.str[0] == 'P'].lineage.nunique()
cfp_count = c_count + f_count + p_count
c_samples = list(mgdata[mgdata.lineage.str[0] == 'C'].lineage.unique())
e_samples = list(mgdata[mgdata.lineage.str[0] == 'E'].lineage.unique())
f_samples = list(mgdata[mgdata.lineage.str[0] == 'F'].lineage.unique())
p_samples = list(mgdata[mgdata.lineage.str[0] == 'P'].lineage.unique())
cfp_samples = c_samples + f_samples + p_samples

/Users/timothywannier/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,35,71) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/timothywannier/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,35,71) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mgdata

,UID,position,lineage,REF,ALT,DP,AF,gene
0,28ee39d1,84,C-C3-1,A,NaN,103.0,0.0,NaN
2,28ee39d1,84,E-C1-1,A,NaN,30.0,0.0,NaN
3,28ee39d1,84,C-C4-1,A,G,59.0,1.0,NaN
4,28ee39d1,84,C-A4-2,A,NaN,50.0,0.0,NaN
5,28ee39d1,84,C-E3-1,A,NaN,75.0,0.0,NaN
6,28ee39d1,84,F-E6-2,A,NaN,69.0,0.0,NaN
7,28ee39d1,84,P-2-1,A,NaN,57.0,0.0,NaN
8,28ee39d1,84,P-13-1,A,NaN,27.0,0.0,NaN
9,28ee39d1,84,C-F3-1,A,NaN,41.0,0.0,NaN
10,28ee39d1,84,F-G5-1,A,NaN,69.0,0.0,NaN


Figure out a bit of stuff about the variant set....looking for numbers of certain types of variants, read depth, etc.

Instance 1: Number of variant calls with read depth < min_read_depth
Instance 2: Number of variant calls with allele fracgion < min_allele_fraction 

In [4]:
# Set parameters for judging variant call quality. 1. Length of reference, 2. read depth, 3. allele fraction
# Apply more stringent criteria to variants with long references, often which have strings of single nucleotides.
# Also weed out variants with high SRP (uneven coverage on F and R strands)?
long = 6
depth = 6
depth_long = 6
fraction = 0.9
fraction_long = 0.9
#max_srp = 100

# create dataframes that contain only the called variants
mv = mgdata[mgdata.ALT.notnull()]
cv = c3data[c3data.ALT.notnull()]

# report out some statistics
print("total variant calls to MG1655v3: ",mv.UID.count(),
      "\n -- at an average read depth of: ", round(mgdata.DP.mean(),1))
print("variant calls under min read depth: ",mv[mv.DP < depth].UID.count())
print("variant calls under min allele fraction: ",mv[mv.AF < fraction].UID.count())
print("\ntotal variant calls to c321 genbank: ",cv.UID.count(),
      "\n -- at an average read depth of: ", round(c3data.DP.mean(),1))
print("variant calls under min read depth: ",cv[cv.DP < depth].UID.count())
print("variant calls under min allele fraction: ",cv[cv.AF < fraction].UID.count())

# Remove reads under minimal read depth or allele fraction. 
mvc = mv[(((mv.DP >= depth) & (mv.REF.str.len() < long)) | ((mv.DP >= depth_long) & (mv.REF.str.len() > long))) &
         (((mv.AF >= fraction) & (mv.REF.str.len() < long)) | ((mv.AF >= fraction_long) & (mv.REF.str.len() > long)))]       
                 
cvc = cv[(((cv.DP >= depth) & (cv.REF.str.len() < long)) | ((cv.DP >= depth_long) & (cv.REF.str.len() > long))) &
         (((cv.AF >= fraction) & (cv.REF.str.len() < long)) | ((cv.AF >= fraction_long) & (cv.REF.str.len() > long)))]       

print("total MG1655 culled variant calls: ",mvc.UID.count())
print("total C321 culled variant calls: ",cvc.UID.count())

# Separate the culled variant calls by..
# data set: ECNR2.1, C321, C321.∆A, and C321.∆A_F11
# generation: g0 vs g1,100
# alignment: to MG1655 or to C321.deltaA

g0Emvc = mvc[(mvc.lineage.str[3] == '0') & (mvc.lineage.str[0] == 'E')]
g0Cmvc = mvc[(mvc.lineage.str[3] == '0') & (mvc.lineage.str[0] == 'C')]
g0Fmvc = mvc[(mvc.lineage.str[3] == '0') & (mvc.lineage.str[0] == 'F')]
Emvc = mvc[(mvc.lineage.str[3] != '0') & (mvc.lineage.str[0] == 'E')]
Cmvc = mvc[(mvc.lineage.str[3] != '0') & (mvc.lineage.str[0] == 'C')]
Fmvc = mvc[(mvc.lineage.str[3] != '0') & (mvc.lineage.str[0] == 'F')]
Pmvc = mvc[mvc.lineage.str[0] == 'P']
g0Ecvc = cvc[(cvc.lineage.str[3] == '0') & (cvc.lineage.str[0] == 'E')]
g0Ccvc = cvc[(cvc.lineage.str[3] == '0') & (cvc.lineage.str[0] == 'C')]
g0Fcvc = cvc[(cvc.lineage.str[3] == '0') & (cvc.lineage.str[0] == 'F')]
Ecvc = cvc[(cvc.lineage.str[3] != '0') & (cvc.lineage.str[0] == 'E')]
Ccvc = cvc[(cvc.lineage.str[3] != '0') & (cvc.lineage.str[0] == 'C')]
Fcvc = cvc[(cvc.lineage.str[3] != '0') & (cvc.lineage.str[0] == 'F')]
Pcvc = cvc[cvc.lineage.str[0] == 'P']

# print out some statistics
print("\ntotal gen0 MG1655 ENCR2.1 culled variant calls: ",g0Emvc.UID.count())
print("total gen0 MG1655 C321.∆A culled variant calls: ",g0Cmvc.UID.count())
print("total gen0 MG1655 C321.∆A_F11 culled variant calls: ",g0Fmvc.UID.count())
print("\ntotal MG1655 to ENCR2.1 culled variant calls:",Emvc.UID.count()," -- per sample: ",
      round(Emvc.UID.count() / Emvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Emvc.UID.count() / Emvc.lineage.nunique()-g0Emvc.UID.count(),1),
      "\n   total unique variants: ",Emvc.UID.nunique()," -- not found in gen0: ",
      Emvc.UID[~Emvc.UID.isin(g0Emvc.UID)].nunique())
print("total MG1655 to C321 culled variant calls:",Pmvc.UID.count()," -- per sample: ",
      round(Pmvc.UID.count() / Pmvc.lineage.nunique(),1))
print("total MG1655 to C321.∆A culled variant calls:",Cmvc.UID.count()," -- per sample: ",
      round(Cmvc.UID.count() / Cmvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Cmvc.UID.count() / Cmvc.lineage.nunique()-g0Cmvc.UID.count(),1),
      "\n   total unique variants: ",Cmvc.UID.nunique()," -- not found in gen0: ",
      Cmvc.UID[~Cmvc.UID.isin(g0Cmvc.UID)].nunique())
print("total MG1655 to C321.∆A_F11 culled variant calls:",Fmvc.UID.count()," -- per sample: ",
      round(Fmvc.UID.count() / Fmvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Fmvc.UID.count() / Fmvc.lineage.nunique()-g0Fmvc.UID.count(),1),
      "\n   total unique variants: ",Fmvc.UID.nunique()," -- not found in gen0: ",
      Fmvc.UID[~Fmvc.UID.isin(g0Fmvc.UID)].nunique())
print("\ntotal gen0 C321 ENCR2.1 culled variant calls: ",g0Ecvc.UID.count())
#print("total gen0 C321 C321 culled variant calls: ",g0Pcvc.UID.count())
print("total gen0 C321 C321.∆A culled variant calls: ",g0Ccvc.UID.count())
print("total gen0 C321 C321.∆A_F11 culled variant calls: ",g0Fcvc.UID.count())
print("\ntotal C321 to ENCR2.1 culled variant calls:",Ecvc.UID.count()," -- per sample: ",
      round(Ecvc.UID.count() / Ecvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Ecvc.UID.count() / Ecvc.lineage.nunique()-g0Ecvc.UID.count(),1),
      "\n   total unique variants: ",Ecvc.UID.nunique()," -- not found in gen0: ",
      Ecvc.UID[~Ecvc.UID.isin(g0Ecvc.UID)].nunique())
print("total C321 to C321 culled variant calls:",Pcvc.UID.count()," -- per sample: ",
      round(Pcvc.UID.count() / Pcvc.lineage.nunique(),1))
print("total C321 to C321.∆A culled variant calls:",Ccvc.UID.count()," -- per sample: ",
      round(Ccvc.UID.count() / Ccvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Ccvc.UID.count() / Ccvc.lineage.nunique()-g0Ccvc.UID.count(),1),
      "\n   total unique variants: ",Ccvc.UID.nunique()," -- not found in gen0: ",
      Ccvc.UID[~Ccvc.UID.isin(g0Ccvc.UID)].nunique())
print("total C321 to C321.∆A_F11 culled variant calls:",Fcvc.UID.count()," -- per sample: ",
      round(Fcvc.UID.count() / Fcvc.lineage.nunique(),1),
      " -- and in excess of gen0: ",
      round(Fcvc.UID.count() / Fcvc.lineage.nunique()-g0Fcvc.UID.count(),1),
      "\n   total unique variants: ",Fcvc.UID.nunique()," -- not found in gen0: ",
      Fcvc.UID[~Fcvc.UID.isin(g0Fcvc.UID)].nunique())

total variant calls to MG1655v3:  68415 
 -- at an average read depth of:  52.8
variant calls under min read depth:  2364
variant calls under min allele fraction:  1345

total variant calls to c321 genbank:  33225 
 -- at an average read depth of:  53.2
variant calls under min read depth:  812
variant calls under min allele fraction:  1089
total MG1655 culled variant calls:  64639
total C321 culled variant calls:  31254

total gen0 MG1655 ENCR2.1 culled variant calls:  28
total gen0 MG1655 C321.∆A culled variant calls:  685
total gen0 MG1655 C321.∆A_F11 culled variant calls:  714

total MG1655 to ENCR2.1 culled variant calls: 1919  -- per sample:  68.5  -- and in excess of gen0:  40.5 
   total unique variants:  818  -- not found in gen0:  796
total MG1655 to C321 culled variant calls: 20671  -- per sample:  738.2
total MG1655 to C321.∆A culled variant calls: 19251  -- per sample:  740.4  -- and in excess of gen0:  55.4 
   total unique variants:  1580  -- not found in gen0:  897
total

Remove gen0 variants from the evolved populations.  Then apply a threshold cutoff to look at interesting variants.  Only look at genes his multiple times across biological replicates.

**We haven't yet sequenced c321_prfA+ generation zero, so for now, use c321.∆A as a stand in for this variant. Add in hand-called list of ancestral mutations here..

(yaiO is a string of 9 Gs, so it's just miscalled)

In [5]:
# set threshold for minimum number of mutations needed to meet significance.
threshold = 5
positional_threshold = 3

# remove UIDs that are present in the full variant list of the generation zero population 
g0_E_ALL = mv[(mv.lineage.str[3] == '0') & (mv.lineage.str[0] == 'E')]
g0_C_ALL = mv[(mv.lineage.str[3] == '0') & (mv.lineage.str[0] == 'C')]
g0_F_ALL = mv[(mv.lineage.str[3] == '0') & (mv.lineage.str[0] == 'F')]

EmvcNEW = Emvc[~Emvc.UID.isin(g0_E_ALL.UID)]
PmvcNEW = Pmvc[~Pmvc.UID.isin(g0_C_ALL.UID)] # use C321.dA g0 until c321 g0 is sequenced
CmvcNEW = Cmvc[~Cmvc.UID.isin(g0_C_ALL.UID)]
FmvcNEW = Fmvc[~Fmvc.UID.isin(g0_F_ALL.UID)] 

# crate output matrix of ancestral variants for each population
ancestral_variants = pd.concat([Emvc[Emvc.UID.isin(g0_E_ALL.UID)].gene.value_counts(),
                                Pmvc[Pmvc.UID.isin(g0_C_ALL.UID)].gene.value_counts(),
                                Cmvc[Cmvc.UID.isin(g0_C_ALL.UID)].gene.value_counts(),
                                Fmvc[Fmvc.UID.isin(g0_F_ALL.UID)].gene.value_counts()], axis=1)
ancestral_variants.columns = ['ECNR2','C321','C321_dA','C321_dA_F11']

# create list of new variants and filter for variants that occur at or above threshold
new_hits = pd.concat([EmvcNEW.gene.value_counts(),PmvcNEW.gene.value_counts(),
                      CmvcNEW.gene.value_counts(),FmvcNEW.gene.value_counts()], axis=1)
new_hits.columns = ['ECNR2','C321','C321_dA','C321_dA_F11']
threshold_hits = new_hits[(new_hits.ECNR2 >= threshold) | (new_hits.C321 >= threshold) | 
                          (new_hits.C321_dA >= threshold) | (new_hits.C321_dA_F11 >= threshold)]

# print out some statistics
print('removed',len(ancestral_variants), 'ancestral hits from evolved populations.\n',)
print('number of genes hit over threshold in evolved \nECNR2.1 populations:',
      threshold_hits[threshold_hits.ECNR2 >= threshold].ECNR2.count(),
      ' --C321 poipulations:',
      threshold_hits[threshold_hits.C321 >= threshold].C321.count(),
      ' --C321.∆A poipulations:',
      threshold_hits[threshold_hits.C321_dA >= threshold].C321_dA.count(),
      ' --C321.∆A_F11 populations:',
      threshold_hits[threshold_hits.C321_dA_F11 >= threshold].C321_dA_F11.count(),'\n')
print('genes hit: \n',threshold_hits)

removed 632 ancestral hits from evolved populations.

number of genes hit over threshold in evolved 
ECNR2.1 populations: 20  --C321 poipulations: 46  --C321.∆A poipulations: 36  --C321.∆A_F11 populations: 22 

genes hit: 
        ECNR2  C321  C321_dA  C321_dA_F11
aceE     NaN   1.0      7.0          2.0
aceF     1.0   5.0      NaN          NaN
acnA     NaN   NaN      5.0          2.0
aer      1.0   NaN      5.0          2.0
astD     NaN   5.0      NaN          NaN
cmtA     NaN   NaN      NaN         10.0
cyaA     NaN   4.0      5.0          3.0
dcp      2.0   NaN      NaN          8.0
dcuS    23.0   NaN      NaN          1.0
dsdC     NaN   NaN      5.0          NaN
efeO     NaN   NaN      NaN         11.0
fdnG     NaN   NaN      5.0          NaN
fimH     1.0   2.0      3.0         19.0
fis      NaN   7.0     19.0          NaN
flgK     NaN  27.0      1.0          NaN
flu     31.0   NaN     11.0         22.0
folA     NaN  22.0     20.0          4.0
folM     NaN  10.0      1.0          N

Further analyze threshold variants, specifically looking for mutations in long strings of single nucleotides that could be sequencing errors.



# Building up a phylogenetic tree for each experiment

In [6]:
# create a dataframe containing all UIDs for each sample
i = 0
all_Emutations = pd.DataFrame()
for sample in EmvcNEW.lineage.unique():
    Emutations = pd.DataFrame()
    Emutations = pd.DataFrame(EmvcNEW[EmvcNEW.lineage == sample].UID)
    Emutations = Emutations.set_index(pd.Series(range(len(Emutations))))
    Emutations.columns = [sample]
    all_Emutations = pd.concat([all_Emutations,Emutations],axis=1)
    i += 1
    
i = 0
all_Pmutations = pd.DataFrame()
for sample in PmvcNEW.lineage.unique():
    Pmutations = pd.DataFrame()
    Pmutations = pd.DataFrame(PmvcNEW[PmvcNEW.lineage == sample].UID)
    Pmutations = Pmutations.set_index(pd.Series(range(len(Pmutations))))
    Pmutations.columns = [sample]
    all_Pmutations = pd.concat([all_Pmutations,Pmutations],axis=1)
    i += 1
    
i = 0
all_Cmutations = pd.DataFrame()
for sample in CmvcNEW.lineage.unique():
    Cmutations = pd.DataFrame()
    Cmutations = pd.DataFrame(CmvcNEW[CmvcNEW.lineage == sample].UID)
    Cmutations = Cmutations.set_index(pd.Series(range(len(Cmutations))))
    Cmutations.columns = [sample]
    all_Cmutations = pd.concat([all_Cmutations,Cmutations],axis=1)
    i += 1

i = 0
all_Fmutations = pd.DataFrame()
for sample in FmvcNEW.lineage.unique():
    Fmutations = pd.DataFrame()
    Fmutations = pd.DataFrame(FmvcNEW[FmvcNEW.lineage == sample].UID)
    Fmutations = Fmutations.set_index(pd.Series(range(len(Fmutations))))
    Fmutations.columns = [sample]
    all_Fmutations = pd.concat([all_Fmutations,Fmutations],axis=1)
    i += 1

# create a comparative dataframe that contains the number of shared UIDs for each sample pair
compare_Emutations = pd.DataFrame(index=all_Emutations.columns,columns=all_Emutations.columns)
for column in compare_Emutations.columns:
    for row in compare_Emutations.columns:
        testrow = all_Emutations[row].dropna()
        testcolumn = all_Emutations[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_Emutations[column][row] = count[True]

compare_Pmutations = pd.DataFrame(index=all_Pmutations.columns,columns=all_Pmutations.columns)
for column in compare_Pmutations.columns:
    for row in compare_Pmutations.columns:
        testrow = all_Pmutations[row].dropna()
        testcolumn = all_Pmutations[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_Pmutations[column][row] = count[True]

compare_Cmutations = pd.DataFrame(index=all_Cmutations.columns,columns=all_Cmutations.columns)
for column in compare_Cmutations.columns:
    for row in compare_Cmutations.columns:
        testrow = all_Cmutations[row].dropna()
        testcolumn = all_Cmutations[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_Cmutations[column][row] = count[True]

compare_Fmutations = pd.DataFrame(index=all_Fmutations.columns,columns=all_Fmutations.columns)
for column in compare_Fmutations.columns:
    for row in compare_Fmutations.columns:
        testrow = all_Fmutations[row].dropna()
        testcolumn = all_Fmutations[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_Fmutations[column][row] = count[True]
       
compare_Emutations = compare_Emutations.sort_index()
compare_Emutations = compare_Emutations.sort_index(axis = 1)
compare_Pmutations = compare_Pmutations.sort_index()
compare_Pmutations = compare_Pmutations.sort_index(axis = 1)
compare_Cmutations = compare_Cmutations.sort_index()
compare_Cmutations = compare_Cmutations.sort_index(axis = 1)
compare_Fmutations = compare_Fmutations.sort_index()
compare_Fmutations = compare_Fmutations.sort_index(axis = 1)
compare_Cmutations
        

,C-A4-1,C-A4-2,C-B3-1,C-B3-2,C-B4-1,C-B4-2,C-C3-1,C-C3-2,C-C4-1,C-C4-2,C-D3-1,C-D3-2,C-D4-1,C-D4-2,C-E3-1,C-E3-2,C-E4-1,C-E4-2,C-F3-1,C-F3-2,C-F4-1,C-F4-2,C-G3-1,C-G3-2,C-G4-1,C-G4-2
C-A4-1,74,72,NaN,NaN,NaN,NaN,1,3,NaN,NaN,NaN,NaN,NaN,2,3,1,NaN,NaN,NaN,1,1,1,NaN,NaN,NaN,NaN
C-A4-2,71,74,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,2,2,1,NaN,NaN,NaN,1,1,1,NaN,NaN,NaN,NaN
C-B3-1,NaN,NaN,59,34,1,1,1,1,1,2,36,36,2,2,1,1,NaN,NaN,2,2,3,3,2,2,NaN,NaN
C-B3-2,NaN,NaN,34,56,1,1,1,1,1,1,34,35,2,2,1,1,NaN,NaN,1,1,3,3,1,1,NaN,NaN
C-B4-1,NaN,NaN,1,1,33,28,1,1,12,12,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C-B4-2,NaN,NaN,1,1,28,46,1,1,13,13,1,1,1,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C-C3-1,1,1,1,1,1,1,62,41,1,1,1,1,1,1,24,25,NaN,NaN,1,1,1,1,1,1,NaN,NaN
C-C3-2,3,3,1,1,1,1,41,79,1,1,1,1,1,1,26,25,NaN,NaN,2,2,1,1,1,1,NaN,NaN
C-C4-1,NaN,NaN,1,1,12,13,1,1,42,32,1,1,1,1,1,1,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,NaN
C-C4-2,NaN,NaN,2,1,12,13,1,1,32,49,1,2,1,1,1,1,NaN,NaN,1,NaN,1,1,NaN,NaN,NaN,NaN


Begin to analyze what's going on with some of the very frequently called hits. [positionS FROM MG1655v3]
Sort by UID. UIDs hit very frequently in patterns across the population may have been present in the founder lines.

ECNR2.1 seems to especially have a few founder mutations, and interestingly well A1, uniquely, does not share the two most abundant mutations. Those two mutations are present in every other evolved line, but not the founder line, and are as follows: POS 1287826 - Gene purU AND POS 573440 - Gene rusA. Then the third most abundant mutation is additionally not shared by well G1, and only present in one of two colonies from well F2, and is as follows: POS 4351093 - Gene dcuS. Then the fourth most abundant mutation is additionally not shared by wells C2 and H2: POS 2248780 - Gene yeiE.

P populations have some variants that appear to be founders: POS 2288554, 53006, 2995224, 1139782, 1711495, 588492, 4585812, 2614611, 1423525, 2741231, 3309007, 4566573, 1995714, 66624, 1946418, 49764, 1873312, 384055, 2416764

Discount founder mutations from population branch points.

F population groups:  
wells E5/E6/F5/G5/G6  
wells B5/B6/C6/H5  
  
C population groups:  
wells B3/D3  
wells B4/C4  
wells C3/E3  
wells D4/F4  
wells E4/G4  
wells F3/G3  
  
P populatin groups:  
wells 5/6/11  
wells 5/6  
wells 8/13
  
Poor read quality alleles found in all four populations: POS 1594124 and 1594248 - Gene yneO, POS 380012 - Gene yaiO, POS 380012 - Gene mtr, POS 1050297 and 1050308 in gene insB1 (a heavily mutated transposable element), 

In [7]:
founder_groups = [['F-E5-1','F-E5-2','F-E6-1','F-E6-2','F-F5-1','F-F5-2','F-G5-1','F-G5-2','F-G6-1','F-G6-2'],
                  ['F-B5-1','F-B5-2','F-B6-1','F-B6-2','F-C6-1','F-C6-2','F-H5-1','F-H5-2'],
                  ['C-B3-1','C-B3-2','C-D3-1','C-D3-2'],['C-B4-1','C-B4-2','C-C4-1','C-C4-2'],
                  ['C-C3-1','C-C3-2','C-E3-1','C-E3-2'],['C-D4-1','C-D4-2','C-F4-1','C-F4-2'],
                  ['C-E4-1','C-E4-2','C-G4-1','C-G4-2'],['C-F3-1','C-F3-2','C-G3-1','C-G3-2'],
                  ['P-5-1','P-6-1','P-11-1','P-5-2','P-6-2','P-11-2'],
                  ['P-5-1','P-6-1','P-5-2','P-6-2'],
                  ['P-8-1','P-8-2','P-13-1','P-13-2']]

# remove all but two variants from mutations that occur in all members of any founder group
EmvcNEWc = EmvcNEW.copy()
PmvcNEWc = PmvcNEW.copy()
CmvcNEWc = CmvcNEW.copy()
FmvcNEWc = FmvcNEW.copy()
for g in founder_groups:
    if g[0][0] == 'C':
        for v in CmvcNEWc[CmvcNEWc.lineage == g[0]].UID.tolist():
            present = True
            for s in g:
                if v not in CmvcNEWc[CmvcNEWc.lineage == s].UID.tolist():
                    present = False
            if present:
                for s in g[2:]:
                    CmvcNEWc = CmvcNEWc[~((CmvcNEWc.lineage == s) & (CmvcNEWc.UID == v))]
    elif g[0][0] == 'F':
        for v in FmvcNEWc[FmvcNEWc.lineage == g[0]].UID.tolist():
            present = True
            for s in g:
                if v not in FmvcNEWc[FmvcNEWc.lineage == s].UID.tolist():
                    present = False
            if present:
                for s in g[2:]:
                    FmvcNEWc = FmvcNEWc[~((FmvcNEWc.lineage == s) & (FmvcNEWc.UID == v))]
    elif g[0][0] == 'P':
        for v in PmvcNEWc[PmvcNEWc.lineage == g[0]].UID.tolist():
            present = True
            for s in g:
                if v not in PmvcNEWc[PmvcNEWc.lineage == s].UID.tolist():
                    present = False
            if present:
                for s in g[2:]:
                    PmvcNEWc = PmvcNEWc[~((PmvcNEWc.lineage == s) & (PmvcNEWc.UID == v))]
            

problem_POS = [1287826, 573440, 4351093, 2248780, 1594124, 1594248, 1050297, 1050308, 380012, 3304806, 2288554, 53006, 
               2995224, 1139782, 1711495, 588492, 4585812, 2614611, 1423525, 2741231, 3309007, 4566573, 1995714, 66624, 
               1946418, 49764, 1873312, 384055, 2416764]

# remove problem POS from variant lists
EmvcNEWc = EmvcNEWc[~EmvcNEWc.position.isin(problem_POS)]
PmvcNEWc = PmvcNEWc[~PmvcNEWc.position.isin(problem_POS)]
CmvcNEWc = CmvcNEWc[~CmvcNEWc.position.isin(problem_POS)]
FmvcNEWc = FmvcNEWc[~FmvcNEWc.position.isin(problem_POS)]

# create a dataframe containing all UIDs for each sample
i = 0
all_EmutationsC = pd.DataFrame()
for sample in EmvcNEWc.lineage.unique():
    Emutations = pd.DataFrame()
    Emutations = pd.DataFrame(EmvcNEWc[EmvcNEWc.lineage == sample].UID)
    Emutations = Emutations.set_index(pd.Series(range(len(Emutations))))
    Emutations.columns = [sample]
    all_EmutationsC = pd.concat([all_EmutationsC,Emutations],axis=1)
    i += 1

i = 0
all_PmutationsC = pd.DataFrame()
for sample in PmvcNEWc.lineage.unique():
    Pmutations = pd.DataFrame()
    Pmutations = pd.DataFrame(PmvcNEWc[PmvcNEWc.lineage == sample].UID)
    Pmutations = Pmutations.set_index(pd.Series(range(len(Pmutations))))
    Pmutations.columns = [sample]
    all_PmutationsC = pd.concat([all_PmutationsC,Pmutations],axis=1)
    i += 1
    
i = 0
all_CmutationsC = pd.DataFrame()
for sample in CmvcNEWc.lineage.unique():
    Cmutations = pd.DataFrame()
    Cmutations = pd.DataFrame(CmvcNEWc[CmvcNEWc.lineage == sample].UID)
    Cmutations = Cmutations.set_index(pd.Series(range(len(Cmutations))))
    Cmutations.columns = [sample]
    all_CmutationsC = pd.concat([all_CmutationsC,Cmutations],axis=1)
    i += 1

i = 0
all_FmutationsC = pd.DataFrame()
for sample in FmvcNEWc.lineage.unique():
    Fmutations = pd.DataFrame()
    Fmutations = pd.DataFrame(FmvcNEWc[FmvcNEWc.lineage == sample].UID)
    Fmutations = Fmutations.set_index(pd.Series(range(len(Fmutations))))
    Fmutations.columns = [sample]
    all_FmutationsC = pd.concat([all_FmutationsC,Fmutations],axis=1)
    i += 1

# create a comparative dataframe that contains the number of shared UIDs for each sample pair
compare_EmutationsC = pd.DataFrame(index=all_EmutationsC.columns,columns=all_EmutationsC.columns)
for column in compare_EmutationsC.columns:
    for row in compare_EmutationsC.columns:
        testrow = all_EmutationsC[row].dropna()
        testcolumn = all_EmutationsC[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_EmutationsC[column][row] = count[True]

compare_PmutationsC = pd.DataFrame(index=all_PmutationsC.columns,columns=all_PmutationsC.columns)
for column in compare_PmutationsC.columns:
    for row in compare_PmutationsC.columns:
        testrow = all_PmutationsC[row].dropna()
        testcolumn = all_PmutationsC[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_PmutationsC[column][row] = count[True]

compare_CmutationsC = pd.DataFrame(index=all_CmutationsC.columns,columns=all_CmutationsC.columns)
for column in compare_CmutationsC.columns:
    for row in compare_CmutationsC.columns:
        testrow = all_CmutationsC[row].dropna()
        testcolumn = all_CmutationsC[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_CmutationsC[column][row] = count[True]

compare_FmutationsC = pd.DataFrame(index=all_FmutationsC.columns,columns=all_FmutationsC.columns)
for column in compare_FmutationsC.columns:
    for row in compare_FmutationsC.columns:
        testrow = all_FmutationsC[row].dropna()
        testcolumn = all_FmutationsC[column].dropna()
        test = testrow.isin(testcolumn)
        count = test.value_counts()
        if True in count:
            compare_FmutationsC[column][row] = count[True]
       
compare_EmutationsC = compare_EmutationsC.sort_index()
compare_EmutationsC = compare_EmutationsC.sort_index(axis = 1)
compare_PmutationsC = compare_PmutationsC.sort_index()
compare_PmutationsC = compare_PmutationsC.sort_index(axis = 1)
compare_CmutationsC = compare_CmutationsC.sort_index()
compare_CmutationsC = compare_CmutationsC.sort_index(axis = 1)
compare_FmutationsC = compare_FmutationsC.sort_index()
compare_FmutationsC = compare_FmutationsC.sort_index(axis = 1)

all_UIDs = pd.concat([all_EmutationsC.stack().value_counts(),all_PmutationsC.stack().value_counts(),
           all_CmutationsC.stack().value_counts(),all_FmutationsC.stack().value_counts()],axis = 1)
all_UIDs.columns = ['ECNR2','C321','C321_dA','C321_dA_F11']
print("Culled gene variant UIDs that occur most frequently include: \n",
      all_UIDs[(all_UIDs.ECNR2 > 5) | (all_UIDs.C321 > 5) | (all_UIDs.C321_dA > 5) | (all_UIDs.C321_dA_F11 > 5)].sort_values(by = 'ECNR2',ascending = False))


compare_PmutationsC

Culled gene variant UIDs that occur most frequently include: 
           ECNR2  C321  C321_dA  C321_dA_F11
245949d6   17.0   7.0      8.0          4.0
9bbad34b   11.0   6.0      1.0          6.0
b7b65239   10.0   NaN      2.0          9.0
1e2dcc3d    8.0   NaN      NaN          NaN
fde7e5b8    8.0   NaN      NaN          7.0
2ee2dd00    7.0   NaN      NaN          NaN
54a0e7ce    7.0   1.0      9.0          4.0
73812a16    7.0   2.0      2.0          8.0
0ee8f534    6.0   NaN      NaN          NaN
17131e16    6.0   NaN      NaN          NaN
5f36ac54    6.0   NaN      NaN          4.0
b062d7f8    6.0   5.0      6.0          NaN
2953bf45    2.0   5.0      NaN          6.0
206b7322    NaN   8.0      NaN          NaN
9b1974cd    NaN   NaN      5.0         14.0
e5e8bfc2    NaN   7.0      4.0          2.0
f1e9f70d    NaN   NaN      2.0          6.0


,P-1-1,P-1-2,P-10-1,P-10-2,P-11-1,P-11-2,P-12-1,P-12-2,P-13-1,P-14-1,P-14-2,P-2-1,P-2-2,P-3-1,P-3-2,P-4-1,P-4-2,P-5-1,P-5-2,P-6-1,P-6-2,P-7-1,P-7-2,P-8-1,P-8-2,P-9-1,P-9-2
P-1-1,53,49,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P-1-2,49,53,NaN,NaN,NaN,NaN,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,1,NaN,1,1,NaN,NaN,NaN,NaN
P-10-1,NaN,NaN,59,31,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,NaN,NaN,NaN,NaN,2,2,NaN,NaN,1,1
P-10-2,NaN,NaN,31,52,1,1,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,NaN,NaN,NaN,NaN,2,2,NaN,NaN,1,1
P-11-1,NaN,NaN,1,1,51,26,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,1,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,2,NaN
P-11-2,NaN,NaN,1,1,26,49,NaN,1,NaN,NaN,NaN,2,2,NaN,NaN,1,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
P-12-1,1,1,NaN,NaN,NaN,NaN,70,41,NaN,NaN,NaN,NaN,1,1,NaN,1,1,1,NaN,1,NaN,1,1,NaN,NaN,NaN,NaN
P-12-2,NaN,NaN,NaN,NaN,NaN,1,41,82,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,1,1
P-13-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,NaN,NaN
P-14-1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,160,70,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN


In [59]:
threshold_hitsC = pd.concat([EmvcNEWc.gene.value_counts(),PmvcNEWc.gene.value_counts(),
                             CmvcNEWc.gene.value_counts(),FmvcNEWc.gene.value_counts()], axis=1)
threshold_hitsC.columns = ['ECNR2','C321','C321_dA','C321_dA_F11']
threshold_hitsC = threshold_hitsC[(threshold_hitsC.ECNR2 >= threshold) | (threshold_hitsC.C321 >= threshold) | 
                                  (threshold_hitsC.C321_dA >= threshold) | (threshold_hitsC.C321_dA_F11 >= threshold)]

intergenic_hitsC = pd.concat([EmvcNEWc[EmvcNEWc.gene.isnull()].position.value_counts(),
                              PmvcNEWc[PmvcNEWc.gene.isnull()].position.value_counts(),
                              CmvcNEWc[CmvcNEWc.gene.isnull()].position.value_counts(),
                              FmvcNEWc[FmvcNEWc.gene.isnull()].position.value_counts()], axis=1)
intergenic_hitsC.columns = ['ECNR2','C321','C321_dA','C321_dA_F11']
intergenic_hitsC = intergenic_hitsC[(intergenic_hitsC.ECNR2 >= positional_threshold) | 
                                    (intergenic_hitsC.C321 >= positional_threshold) | 
                                    (intergenic_hitsC.C321_dA >= positional_threshold) |
                                    (intergenic_hitsC.C321_dA_F11 >= positional_threshold)]

print('\nnumber of genes hit at or over threshold in evolved \nECNR2.1 populations:',
      threshold_hitsC[threshold_hitsC.ECNR2 >= threshold].ECNR2.count(),
      '\nC321 poipulations:',
      threshold_hitsC[threshold_hitsC.C321 >= threshold].C321.count(),
      '\nC321.∆A poipulations:',
      threshold_hitsC[threshold_hitsC.C321_dA >= threshold].C321_dA.count(),
      '\nC321.∆A_F11 populations:',
      threshold_hitsC[threshold_hitsC.C321_dA_F11 >= threshold].C321_dA_F11.count(),'\n')

writer = pd.ExcelWriter('gene_variants.xlsx', engine='xlsxwriter')
ancestral_variants.to_excel(writer,sheet_name='ancestral')
threshold_hits.to_excel(writer,sheet_name='variants_base')
threshold_hitsC.to_excel(writer,sheet_name='variants_culled')
writer.save()

print('genes hit: \n',threshold_hitsC)
print('\nintergenic hits: \n',intergenic_hitsC)



number of genes hit at or over threshold in evolved 
ECNR2.1 populations: 15 
C321 poipulations: 26 
C321.∆A poipulations: 18 
C321.∆A_F11 populations: 14 

genes hit: 
       ECNR2  C321  C321_dA  C321_dA_F11
aceE    NaN   1.0      7.0          2.0
aceF    1.0   5.0      NaN          NaN
astD    NaN   5.0      NaN          NaN
fdnG    NaN   NaN      5.0          NaN
fimH    1.0   2.0      3.0         19.0
fis     NaN   7.0     11.0          NaN
flu    31.0   NaN      9.0         22.0
folA    NaN  20.0     10.0          4.0
folM    NaN   8.0      1.0          NaN
glrR    NaN   6.0      NaN          NaN
gltB    NaN   5.0      4.0          NaN
kup     1.0   5.0      3.0          NaN
leuP    7.0   1.0      9.0          4.0
leuQ    2.0   NaN      3.0          5.0
leuV    8.0   NaN      NaN          7.0
malK    NaN   NaN      5.0          2.0
mdtJ    NaN   NaN      2.0          6.0
mutS    NaN   5.0      NaN          NaN
narX    5.0   NaN      2.0          NaN
nfrA    NaN   5.0      NaN   

In [56]:
CmvcNEWc[CmvcNEWc.gene.isnull()].position.value_counts()

49766      4
2057669    3
1263627    3
2168525    2
4058263    2
4094415    2
3060764    2
2498548    2
84         2
1212080    2
1673789    2
3915259    2
2765411    2
3700498    2
1351189    2
830697     2
3931176    2
3659139    2
2877939    2
1097138    2
3234679    2
4037496    2
3367768    2
1097030    2
915240     1
3774258    1
3384481    1
3991086    1
3754521    1
1151486    1
107544     1
3841840    1
2819222    1
311472     1
2068207    1
1502224    1
755087     1
1754636    1
3828745    1
780976     1
1818543    1
1310401    1
4352433    1
4153616    1
2304635    1
335221     1
945780     1
3925800    1
841585     1
401135     1
3239660    1
4037093    1
1214175    1
1270159    1
4153595    1
2229198    1
1237323    1
2788803    1
1496700    1
34111      1
4168472    1
2222128    1
2718645    1
4281699    1
Name: position, dtype: int64

In [74]:
mgdata[(mgdata.position==3815879)&~(mgdata.ALT.isnull())]

,UID,position,lineage,REF,ALT,DP,AF,gene
430400,166a1092,3815879,C-G4-1,TCC,TCCC,82.0,0.961538,rph
430410,166a1092,3815879,C-G4-2,TCC,TCCC,70.0,0.909091,rph
430432,166a1092,3815879,P-13-2,TCC,TCCC,22.0,0.863636,rph
430448,166a1092,3815879,P-13-1,TCC,TCCC,21.0,1.000000,rph
430449,166a1092,3815879,F-C6-2,TCC,TC,86.0,0.965116,rph
430453,166a1092,3815879,C-E4-1,TCC,TCCC,36.0,0.914286,rph
430460,166a1092,3815879,W72,TCC,TCCC,33.0,1.000000,rph
430463,166a1092,3815879,C-E4-2,TCC,TCCC,81.0,0.950000,rph
430467,166a1092,3815879,P-8-2,TCC,TCCC,33.0,0.969697,rph
430470,166a1092,3815879,W62,TCC,TCCC,20.0,0.950000,rph


In [67]:
mgdata[(mgdata.position<3815987)&(mgdata.position>3815753)&~(mgdata.ALT.isnull())].position.value_counts()

3815863    84
3815800    48
3815810    24
3815879    12
3815820     8
3815813     6
3815844     4
3815916     2
3815907     2
3815823     2
3815754     2
3815943     2
3815986     1
3815949     1
Name: position, dtype: int64

# Check TAG to TAAs

Find every stop codon in E. coli genome (MG1655 version 3 - NC_000913.3), and categorize by Amber-(TAG) / Ochre-(TAA) / Opal-(TGA) (do not consider ncRNA, rRNA, tRNA, tmRNA, or pseudo-genes).  
  
Once this is done, compare the results to a list of TAG positions used in the C321 recoding effort, supplied originally by Gleb, but Aditya took this file and adjusted the positions so that they make sense in MG1655v3. It appears that they missed a codon when they recoded!  
  
Stop locations are recorded as the first position on the sense strand, regardless of if the gene is sense or antisense.  Strand is recorded as 1 for sense or -1 for antisense.

In [10]:
# build a data-collection framework
stopCodons = {'gene':[],'position':[],'strand':[],'codon':[]}
nonProteinGenes = {}

# read-in MG1655v3 sequence file
for record in SeqIO.parse("NC_000913v3.fasta","fasta"):
    MG1655 = record

# read-in genbank file with gene records
for record in SeqIO.parse("NC_000913v3.gb","genbank"): #Use Biopython to get genbank record info for each gene
    
    # record all non protein-coding genes
    for f in record.features:
        if f.type == "ncRNA":
            nonProteinGenes[f.qualifiers["gene"][0]] = "ncRNA"
        if f.type == "rRNA":
            nonProteinGenes[f.qualifiers["gene"][0]] = "rRNA"
        if f.type == "tRNA":
            nonProteinGenes[f.qualifiers["gene"][0]] = "tRNA"
        if f.type == "tmRNA":
            nonProteinGenes[f.qualifiers["gene"][0]] = "tmRNA"
    
    # iterate through all genes again, eliminating pseudo and non-protein coding genes
    for f in record.features:
        if f.type == "gene":
            gene = f.qualifiers["gene"][0]
            
            # check for pseudo genes
            pseudo = False
            try:
                if f.qualifiers["pseudo"]:
                    pseudo = True
            except:
                pass
            
            # if it is a real protein-coding gene, record stop codon and details
            if gene not in nonProteinGenes.keys() and not pseudo:
                strand = f.location.strand
                if strand == -1:
                    stop_loc = f.location.start + 1
                    stop_seq = str(MG1655[stop_loc - 1 : stop_loc + 2].seq.reverse_complement())
                elif strand == 1:
                    stop_loc = f.location.end - 2
                    stop_seq = str(MG1655[stop_loc - 1 : stop_loc + 2].seq)
                else:
                    print('ERROR: no sense detected')
                stopCodons['gene'].append(gene)
                stopCodons['position'].append(stop_loc)
                stopCodons['strand'].append(strand)
                stopCodons['codon'].append(stop_seq)

# print out some statistics
STOP = pd.DataFrame.from_dict(stopCodons)
print("Total stop codon counts in MG1655 gene annotations: \n",STOP.codon.value_counts())
print("\nNon-standard stop codons detected in genes:\n",STOP[~STOP.codon.isin(['TAG','TGA','TAA'])])

# read in stop codon positions from gleb via aditya and compare to TAGs found by my processing
# of the Genbank file.
c321_pos = pd.read_csv("mg1655v3_uag_locations.csv")
matched_TAG = STOP[(STOP.position.isin(c321_pos.coord))|(STOP.position.isin(c321_pos.coord-2))]
unmatched_TAG = STOP[~(STOP.position.isin(matched_TAG.position))&(STOP.codon=='TAG')]

print("\nTAGs matched from c321:\n",matched_TAG.codon.value_counts(),'\n\nTAGs missing from c321:\n',unmatched_TAG,'\n')
missing_TAG = list(x for x in c321_pos.coord if x not in list(matched_TAG.position) and x not in list(matched_TAG.position + 2))
print(len(missing_TAG),'TAGs called in c321, but don\'t correspond to genes in latest annotation:',missing_TAG)

# Add TAGs that were noted in Aditya's file, but missing from my processing into the DataFrame "STOPs"
for p in missing_TAG:
    p = int(p)
    if MG1655[p-1] == 'G':
        stop_loc = p-2
        strand = 1
    elif MG1655[p-1] == 'C':
        stop_loc = p
        strand = -1
    else:
        print('ERROR: position ',p,' was found to be A or T')
    STOP.loc[len(STOP)] = ['TAG','unannotated',stop_loc,strand]

Total stop codon counts in MG1655 gene annotations: 
 TAA    2678
TGA    1175
TAG     287
TTT       1
Name: codon, dtype: int64

Non-standard stop codons detected in genes:
      codon  gene  position  strand
1273   TTT  ralA   1413732       1

TAGs matched from c321:
 TAG    286
Name: codon, dtype: int64 

TAGs missing from c321:
      codon  gene  position  strand
1293   TAG  tfaR   1432984       1 

35 TAGs called in c321, but don't correspond to genes in latest annotation: [239378, 280024, 315244, 340089, 582152, 716388, 1286984, 1381789, 1425980, 1432273, 1529938, 1544384, 1651537, 1803567, 2012001, 2034536, 2169693, 2210944, 2453265, 2471105, 2784529, 2786729, 2787434, 2788238, 2994415, 2995314, 2997689, 3365664, 3583454, 3815863, 3862497, 4500043, 4507463, 4519014, 4569309]


Match called variants with stop codon list. There are two locations where two TAGs abut one another, from genes on opposite strands terminating next to each other or overlapping. This means that in two instances two TAGs are co-counted at positions:                                                      

634747 - TAG on -1 strand (position of first nucleotide in codon)                                    
634744 - TAG on +1 strand                    "                                                      
746723 - TAG on -1 strand                    "                                                      
746724 - TAG on +1 strand                    "                                                      

No mutations were found to any of these doubled up TAGs

frame refers to the first, second, or third base of the codon (positive strand front to back, negative strand back to front)  
  
Note -- There is one mutation to ECNR2 at position 3269600 that happens to a TAG codon, so this mutation is contained in the cfp_TAG dataframe, but with no ALT entries

In [11]:
# create dataframe to hold mutations at STOP positions
mg_STOP = pd.DataFrame()

# find and record all STOP codon variant info.
# record any length variant call for variants called at stop positions
for i in range(0,3):
    df = STOP.copy()
    df['position'] = df['position'].apply(lambda x: x + i)
    dg = mgdata[mgdata.position.isin(df.position)].loc[:,['position','lineage','REF','ALT','DP','AF']]
    df = pd.merge(df,dg,left_on='position',right_on='position')
    df['frame'] = i+1
    mg_STOP = mg_STOP.append(df)
    
# record long variant calls up to 10nt upstream the STOP codon if the variant call overlaps the stop codon
for i in range(-10,0):
    df = STOP.copy()
    df['position'] = df['position'].apply(lambda x: x + i)
    dg = mgdata[mgdata.position.isin(df.position)&(mgdata.REF.str.len() > -i)]
    dg = dg.loc[:,['position','lineage','REF','ALT','DP','AF']]
    df = pd.merge(df,dg,left_on='position',right_on='position')
    df['frame'] = i+1
    mg_STOP = mg_STOP.append(df)

# break STOPs dataframe into sub-sets for CFP and E populations and CFP into TAGs vs TAATGAs
cfp_TAG = mg_STOP[mg_STOP.lineage.str[0].isin(['C','F','P'])&(mg_STOP.codon=='TAG')]
cfp_TAATGA = mg_STOP[(mg_STOP.lineage.str[0].isin(['C','F','P']))&(mg_STOP.codon.isin(['TAA','TGA']))]
e_STOP = mg_STOP[mg_STOP.lineage.str[0]=='E']

# Find reversions to TAGs in CFP populatinos by counting diversion from expected muations
# Note all TAG reversions that don't have zero read depth
underhit_TAG = pd.DataFrame()
for p in cfp_TAG.groupby(['position','ALT']).filter(lambda x: len(x) < cfp_count).position.unique():
    # clean out positions found just because of poor read depth
    hits = cfp_TAG[(cfp_TAG.position==p)&(cfp_TAG.ALT.notnull())].shape[0]
    no_read = cfp_TAG[(cfp_TAG.position==p)&(cfp_TAG.DP.isnull())].shape[0]
    if hits + no_read != cfp_count:
        underhit_TAG = pd.concat([underhit_TAG,cfp_TAG.groupby(['position']).get_group(p)])
underhit_TAG_real = underhit_TAG[(underhit_TAG.ALT.isnull())&(underhit_TAG.DP.notnull())].sort_values(by='position')

# Analyze TGA and TAA mutations among evolving CFP populations
# Divide cfp TGA and TAA mutations into two classes: ancestral (from c321) and new (appeared during evolution)
cfp_TAATGAmut = cfp_TAATGA[(cfp_TAATGA.DP.notnull())&(cfp_TAATGA.ALT.notnull())&(cfp_TAATGA.AF>fraction)]
cfp_TAATGAancestral = cfp_TAATGAmut.groupby(['position','ALT']).filter(lambda x: len(x) >= cfp_count)
cfp_TAATGAnew = cfp_TAATGAmut.groupby(['position','ALT']).filter(lambda x: len(x) < cfp_count)
ancestral = pd.DataFrame(columns=['gene','codon','strand','frame','REF','ALT','count'])
for p in cfp_TAATGAancestral.position.unique():
    ancestral.loc[p,'gene']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].gene.unique()[0]
    ancestral.loc[p,'codon']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].codon.unique()[0]
    ancestral.loc[p,'strand']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].strand.unique()[0]
    ancestral.loc[p,'frame']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].frame.unique()[0]
    ancestral.loc[p,'REF']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].REF.unique()[0]
    ancestral.loc[p,'ALT']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].ALT.unique()[0]
    ancestral.loc[p,'count']=cfp_TAATGAancestral[cfp_TAATGAancestral.position==p].ALT.count()

cfp_TAATGAnew = cfp_TAATGAnew.sort_values(by='position')

# find all stop mutations in ECNR2 lines
e_STOPmut = e_STOP[(e_STOP.ALT.notnull())&(e_STOP.AF>fraction)&(e_STOP.DP.notnull())].sort_values(by='position')

# print out some statistics
p = len(list(matched_TAG.position.unique())+missing_TAG)
s = cfp_TAG.lineage.nunique()
v = cfp_TAG.ALT.count()
# NOTE -- there are two positions with no coverage from position 3269600 that is hit in ECNR2 that need to be 
# subtracted from the count of no-coverage alleles for the numbers to add up.
nc = cfp_TAG[cfp_TAG.DP.isnull()].shape[0]-2
print('There are',p,'TAG variant positions from c321, which times',s,'lineages aligned, should be',p*s
      ,'calls. Less ',nc,'lines with no coverage leaves ',p*s-nc,'calls expected.')
print('Actual number of called variants is:',v,', or',p*s-nc-v,'fewer than predicted.\n')
print('The positions UIDs hit fewer than ',s,' times are: \n',underhit_TAG.position.unique())
print('\nUnderhit lineage positions: ',underhit_TAG_real.position.count(),'\n',underhit_TAG_real)
print('\nThere are',len(cfp_TAATGAancestral)/cfp_count,'ancestral mutations to non-TAG stop codons in C-F-P lines:')
print(ancestral,'\n')
print('There are',len(cfp_TAATGAnew.position),'new mutations to Ochre/Opal codons in C-F-P lines')
print(cfp_TAATGAnew,'\n')
print('There are',len(e_STOPmut.position),'new mutations to stop codons in ECNR2 lines')
print(e_STOPmut,'\n')


There are 321 TAG variant positions from c321, which times 84 lineages aligned, should be 26964 calls. Less  55 lines with no coverage leaves  26909 calls expected.
Actual number of called variants is: 26875 , or 34 fewer than predicted.

The positions UIDs hit fewer than  84  times are: 
 [2012700  568004 1264970 3269602  315244 3365664]

Underhit lineage positions:  34 
       codon         gene  position  strand lineage REF  ALT     DP   AF  frame
18459   TAG  unannotated    315244       1  F-F6-1   G  NaN   83.0  0.0      3
2047    TAG        insF1    568004       1  C-F4-1   G  NaN    1.0  0.0      3
5466    TAG         hemA   1264970       1  P-13-1   G  NaN   24.0  0.0      3
5512    TAG         hemA   1264970       1   P-8-1   G  NaN   56.0  0.0      3
5511    TAG         hemA   1264970       1   P-2-2   G  NaN   12.0  0.0      3
5510    TAG         hemA   1264970       1   P-9-2   G  NaN   21.0  0.0      3
5507    TAG         hemA   1264970       1   P-7-2   G  NaN   17.0  0.0

Scan for frameshift mutations upstream of the end of the gene
    

In [12]:
# record variant calls up to 90nt upstream (but not including) the STOP codon if the variant call indicates 
# a frameshift
buffer = 1000
window = 100

mg_FRAMESHIFT = pd.DataFrame()
for i in range(-window,-1):
    df = STOP.copy()
    # adjust the STOP position
    df['adjustment'] = df['strand'].apply(lambda x: x * i + 1)  
    df['position'] = df['position'] + df['adjustment']
    # pull frameshift mutations
    dg = mvc[mvc.position.isin(df.position) & mvc.ALT.notnull() & (mvc.REF.str.len() != mvc.ALT.str.len())]
    dg = dg.loc[:,['position','lineage','REF','ALT','DP','AF']]
    df = pd.merge(df,dg,left_on='position',right_on='position')
    mg_FRAMESHIFT = mg_FRAMESHIFT.append(df)

# add in a couple of ancestral variants:
mg_FRAMESHIFT.loc[mg_FRAMESHIFT.shape[0]] = ['TAA','rzoR',1423525,1,-58,'P-G0','TCCCCCCG','TCCCCCCCG',50,1]
mg_FRAMESHIFT.loc[mg_FRAMESHIFT.shape[0]] = ['TAG','unannotated',3862515,1,-18,'C-G0','GCC','GC',8,1]

lineages = ['E','C','F','P','CFP']
l_samples = {'E':e_samples,'C':c_samples,'F':f_samples,'P':p_samples,'CFP':cfp_samples}
l_count = {'E':e_count,'C':c_count,'F':f_count,'P':p_count,'CFP':cfp_count}
l_ancestors = {'E':'W72','C':'C-G0','F':'F-G0','P':'P-G0','CFP':'C-G0'}

F_ancestral = pd.DataFrame(columns=['v2gene','v3gene','REF','ALT','offset','codon','codon_alt',
                                    'nt_upstream','new_downstream','strand','population'])
F_reversion = pd.DataFrame(columns=['v2gene','v3gene','REF','ALT','codon','codon_alt','nt_upstream',
                           'new_downstream','strand','population'])

# Populate ancestral DF with info on stop codon and alternate for populations with no reversions
for line in lineages:
    FS = mg_FRAMESHIFT[mg_FRAMESHIFT.lineage.isin(l_samples[line])]
    A_pos = list(mg_FRAMESHIFT[mg_FRAMESHIFT.lineage == l_ancestors[line]].position.unique())
    A_FS = FS[FS.position.isin(A_pos)]
    mg_FS = mgdata[(mgdata.position.isin(A_pos)) & mgdata.lineage.isin(l_samples[line])]
    A_FS = pd.merge(mg_FS,A_FS,how='outer', left_on=['lineage','position','REF','ALT','DP','AF'],
                                            right_on=['lineage','position','REF','ALT','DP','AF'])
    A_FS['position'] = A_FS['position'].astype(int) # prevents floats from messing up processing
    A_FS_norev = A_FS.groupby(['position','ALT']).filter(lambda x: len(x) == l_count[line])
    if line == 'CFP':
        A_FS_norev = A_FS.groupby(['position','ALT']).filter(lambda x: len(x) >= l_count[line])
    A_FS_rev = A_FS.groupby(['position','ALT']).filter(lambda x: len(x) < l_count[line])
    
    for p in A_FS_norev.position.unique():
        F_ancestral.loc[p,'v2gene']=A_FS[A_FS.position==p].gene_x.dropna().unique()[0]
        F_ancestral.loc[p,'v3gene']=A_FS[A_FS.position==p].gene_y.dropna().unique()[0]
        F_ancestral.loc[p,'codon']=A_FS[A_FS.position==p].codon.dropna().unique()[0]
        # get stop codon that the frameshift switches to:
        F_ancestral.loc[p,'REF'] = r = A_FS[A_FS.position==p].REF.unique()[0]
        F_ancestral.loc[p,'ALT'] = a = A_FS[A_FS.position==p].ALT.unique()[0]
        if 'LONG' in r:
            r = 'X' * int(re.findall('\d+', r)[0])
        if 'LONG' in a:
            a = 'X' * int(re.findall('\d+', a)[0])
        F_ancestral.loc[p,'strand']=s=A_FS[A_FS.position==p].strand.dropna().unique()[0]
        u = abs(A_FS[A_FS.position==p].adjustment.dropna().unique()[0])
        F_ancestral.loc[p,'offset']=o=(len(r) - len(a))
        q = int(p - 1 + ((u % 3) + o) * s)
        if s == 1:
            n = MG1655[q : q + buffer]
            aa = n.seq.translate()
            l = len(aa.split('*')[0]) * 3
            F_ancestral.loc[p,'codon_alt'] = str(MG1655[q + l : q + l + 3].seq)
        else:
            u -= 2 # resize u for accurate reporting of upstream bases on reverse strand
            n = MG1655[q - buffer : q]
            aa = n.seq.reverse_complement().translate()
            l = len(aa.split('*')[0]) * 3
            F_ancestral.loc[p,'codon_alt'] = str(MG1655[q - l - 3 : q - l].reverse_complement().seq)
            l += o # resize l for accurate reporting of new downstream bases on reverse strand
        F_ancestral.loc[p,'nt_upstream'] = u
        F_ancestral.loc[p,'new_downstream'] = l + 1
        F_ancestral.loc[p,'population'] = line
        F_ancestral.loc[p,'count'] = A_FS[A_FS.position==p].ALT.count()

# Populate ancestral DF with info on stop codon and alternate for populations with reversions
    for p in A_FS_rev.position.unique():
        i = 0
        alts = A_FS[A_FS.position==p].ALT.unique()
        for alt in alts:
            i += 1
            pos = str(p) + '-' + str(i)
            F_reversion.loc[pos,'v2gene']=A_FS[A_FS.position==p].gene_x.dropna().unique()[0]
            F_reversion.loc[pos,'v3gene']=A_FS[A_FS.position==p].gene_y.dropna().unique()[0]
            F_reversion.loc[pos,'codon']=A_FS[A_FS.position==p].codon.dropna().unique()[0]
            # get stop codon that the frameshift switches to:
            F_reversion.loc[pos,'REF'] = r = A_FS[A_FS.position==p].REF.unique()[0]
            F_reversion.loc[pos,'ALT'] = a = alt
            if 'LONG' in r:
                r = 'X' * int(re.findall('\d+', r)[0])
            if isinstance(a,float):
                a = r
                F_reversion.loc[pos,'count'] = A_FS[A_FS.position==p].ALT.isnull().sum()
            else:
                if 'LONG' in a:
                    a = 'X' * int(re.findall('\d+', a)[0])
                F_reversion.loc[pos,'count'] = A_FS[(A_FS.position==p) & (A_FS.ALT==alt)].shape[0]
            F_reversion.loc[pos,'strand']=s=A_FS[A_FS.position==p].strand.dropna().unique()[0]
            u = abs(A_FS[A_FS.position==p].adjustment.dropna().unique()[0])
            F_reversion.loc[pos,'offset']=o=(len(r) - len(a))
            q = int(p - 1 + ((u % 3) + o) * s)
            if s == 1:
                n = MG1655[q : q + buffer]
                aa = n.seq.translate()
                l = len(aa.split('*')[0]) * 3
                F_reversion.loc[pos,'codon_alt'] = str(MG1655[q + l : q + l + 3].seq)
            else:
                u -= 2 # resize u for accurate reporting of upstream bases on reverse strand
                n = MG1655[q - buffer : q]
                aa = n.seq.reverse_complement().translate()
                l = len(aa.split('*')[0]) * 3
                F_reversion.loc[pos,'codon_alt'] = str(MG1655[q - l - 3 : q - l].reverse_complement().seq)
                l += o # resize l for accurate reporting of new downstream bases on reverse strand
            F_reversion.loc[pos,'nt_upstream'] = u
            F_reversion.loc[pos,'new_downstream'] = l + 1
            F_reversion.loc[pos,'population'] = line

print('The following positions are ancestral frameshifts in CFP populations that maintained complete penetrance',
      'after evolution:\n',F_ancestral,'\n')
print('The following positions are ancestral frameshifts in CFP populations that showed reversion mutations',
      'after evolution:\n',F_reversion,'\n')

/Users/timothywannier/anaconda/lib/python3.6/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


The following positions are ancestral frameshifts in CFP populations that maintained complete penetrance after evolution:
         v2gene       v3gene        REF        ALT offset codon codon_alt  \
183606    cdaR         cdaR        GAT         GT      1   TAG       TAG   
675600    ybeQ         ybeQ        CTG         CG      1   TAG       TGA   
1006412   zapC         zapC        TGG         TG      1   TAA       TGA   
1757751   ldtE         ldtE    CTCTGGT         CT      5   TAG       TAG   
2824495   mltB         mltB        TGT         TT      1   TAG       TAA   
3134810   yghT         yghT        GTG         GG      1   TAG       TAA   
3330353   dacB         dacB        GTT       GTTT     -1   TAG       TGA   
3862523   glvC  unannotated      TGCCC         TC      3   TAG       TAG   
4424542   yjfY         yjfY        GCA         GA      1   TAG       TAA   
4447866   tamB         tamB        ATT         AT      1   TAG       TAA   
4619469   deoA         deoA  LONG:11bp   

In [13]:
# analyze new frameshift mutations that appeared during the evolutionary process
ancestral_positions = list(set([int(x[:-2]) for x in F_reversion.index.tolist()]))
ancestral_positions += F_ancestral.index.tolist()
all_F_new = mg_FRAMESHIFT[~mg_FRAMESHIFT.position.isin(ancestral_positions)]
all_F_new = pd.merge(mgdata[mgdata.position.isin(list(all_F_new.position.unique()))],
                     all_F_new,how='outer',left_on=['lineage','position','REF','ALT','DP','AF'],
                                           right_on=['lineage','position','REF','ALT','DP','AF'])

F_new = pd.DataFrame(columns=['lineages','v2gene','v3gene','REF','ALT','offset','codon','codon_alt','nt_upstream',
                              'new_downstream','strand','count'])
F_new.astype(object)
for p in all_F_new.position.unique():
    i = 0
    alts = all_F_new[all_F_new.position==p].ALT.dropna().unique()
    for alt in alts:
        i += 1
        pos = str(p) + '-' + str(i)
        F_new.set_value(pos,'lineages',list(all_F_new[(all_F_new.position == p) & 
                                                   (all_F_new.ALT == alt)].lineage.unique()))
        F_new.set_value(pos,'v2gene',all_F_new[all_F_new.position == p].gene_x.dropna().unique()[0])
        F_new.set_value(pos,'v3gene',all_F_new[all_F_new.position == p].gene_y.dropna().unique()[0])
        F_new.set_value(pos,'codon',all_F_new[all_F_new.position == p].codon.dropna().unique()[0])
        # get stop codon that the frameshift switches to:
        r = all_F_new[all_F_new.position == p].REF.unique()[0]
        F_new.set_value(pos,'REF',r)
        F_new.set_value(pos,'ALT',alt)
        if 'LONG' in r:
            r = 'X' * int(re.findall('\d+', r)[0])
        if isinstance(alt,float):
            alt = r
            F_new.set_value(pos,'count',all_F_new[all_F_new.position==p].ALT.isnull().sum())
        else:
            if 'LONG' in alt:
                alt = 'X' * int(re.findall('\d+', alt)[0])
            F_new.set_value(pos,'count',all_F_new[(all_F_new.position == p)&(all_F_new.ALT == alt)].shape[0])
        s = all_F_new[all_F_new.position == p].strand.dropna().unique()[0]
        u = abs(all_F_new[all_F_new.position == p].adjustment.dropna().unique()[0])
        o = len(r) - len(alt)
        F_new.set_value(pos,'strand',s)
        F_new.set_value(pos,'offset',o)
        q = int(p - 1 + ((u % 3) + o) * s)
        if s == 1:
            n = MG1655[q : q + buffer]
            aa = n.seq.translate()
            l = len(aa.split('*')[0]) * 3
            F_new.set_value(pos,'codon_alt',str(MG1655[q + l : q + l + 3].seq))
        else:
            u -= 2 # resize u for accurate reporting of upstream bases on reverse strand
            n = MG1655[q - buffer : q]
            aa = n.seq.reverse_complement().translate()
            l = len(aa.split('*')[0]) * 3
            F_new.set_value(pos,'codon_alt',str(MG1655[q - l - 3 : q - l].reverse_complement().seq))
            l += o # resize l for accurate reporting of new downstream bases on reverse strand
        F_new.set_value(pos,'nt_upstream',u)
        F_new.set_value(pos,'new_downstream',l + 1)

print('new frameshift mutations that impact stop codon usage:\n',F_new)

/Users/timothywannier/anaconda/lib/python3.6/site-packages/Bio/Seq.py:2095: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


new frameshift mutations that impact stop codon usage:
                                                     lineages v2gene  \
115797-1                                            [P-10-1]   hofB   
578096-1                                            [E-E2-1]   rrrD   
635369-1                            [P-10-2, C-A4-1, C-A4-2]   ybdM   
821493-1                                            [E-B2-1]   ybhM   
1084580-1                                           [C-D3-1]   efeB   
1189846-1                                   [C-C4-2, F-B6-1]   phoP   
1199759-1                                   [F-G5-2, F-G5-1]   intE   
1417538-1                                           [E-G1-1]   racC   
1639535-1                                           [C-F3-2]   rrrQ   
1660484-1                                           [P-14-2]   ynfE   
1931971-1                   [F-D5-1, F-D5-2, F-G5-1, F-G5-2]   purT   
1967130-1                                           [E-A2-1]   cheY   
2024571-1            

In [14]:
# write out to excel
writer = pd.ExcelWriter('StopCodons.xlsx', engine='xlsxwriter')
matched_TAG.to_excel(writer, sheet_name = 'ancestral TAGs',startcol=0)
unmatched_TAG.to_excel(writer, sheet_name = 'ancestral TAGs',startcol=6)
pd.DataFrame(missing_TAG).to_excel(writer, sheet_name = 'ancestral TAGs',startcol=12)
ancestral.to_excel(writer, sheet_name = 'ancestral TAA-TGAs')
underhit_TAG_real.to_excel(writer, sheet_name = 'CFP TAG reversions')
cfp_TAATGAnew.to_excel(writer, sheet_name = 'new CFP TAA-TGAs')
e_STOPmut.to_excel(writer, sheet_name = 'new E TAA-TAG-TGAs')
F_ancestral.to_excel(writer, sheet_name = 'ancestral FS')
F_reversion.to_excel(writer, sheet_name = 'FS reversions')
F_new.to_excel(writer, sheet_name = 'new FS')
writer.save()

In [15]:
mgdata[(mgdata.position==3862523) & (mgdata.lineage.isin(cfp_samples))]

,UID,position,lineage,REF,ALT,DP,AF,gene
435713,b5a22b1c,3862523,F-B5-1,TGCCC,TC,32.0,1.0,glvC
435714,b5a22b1c,3862523,F-F5-1,TGCCC,TC,83.0,1.0,glvC
435716,b5a22b1c,3862523,F-D5-2,TGCCC,TC,43.0,1.0,glvC
435717,b5a22b1c,3862523,P-9-2,TGCCC,TC,27.0,1.0,glvC
435718,b5a22b1c,3862523,P-3-1,TGCCC,TC,39.0,1.0,glvC
435719,b5a22b1c,3862523,F-G6-2,TGCCC,TC,20.0,1.0,glvC
435720,b5a22b1c,3862523,P-8-1,TGCCC,TC,46.0,1.0,glvC
435721,b5a22b1c,3862523,F-C5-1,TGCCC,TC,27.0,1.0,glvC
435722,b5a22b1c,3862523,F-A6-1,TGCCC,TC,82.0,1.0,glvC
435723,b5a22b1c,3862523,P-13-2,TGCCC,TC,7.0,1.0,glvC


In [16]:
mgdata[(mgdata.gene == 'folA')]

,UID,position,lineage,REF,ALT,DP,AF,gene
5322,5d671889,49787,P-14-1,T,C,64.0,0.984375,folA
5342,5d671889,49787,P-14-2,T,C,30.0,1.000000,folA
5450,76f928a5,49800,C-E4-1,A,T,92.0,0.956522,folA
5452,76f928a5,49800,C-D4-1,A,T,70.0,0.971429,folA
5458,76f928a5,49800,C-E4-2,A,T,111.0,0.981982,folA
5462,76f928a5,49800,C-F4-2,A,T,61.0,0.934426,folA
5469,76f928a5,49800,P-11-2,A,T,22.0,0.090909,folA
5514,76f928a5,49800,C-G4-1,A,T,100.0,0.990000,folA
5523,76f928a5,49800,C-G4-2,A,T,97.0,0.979381,folA
5532,76f928a5,49800,C-F4-1,A,T,8.0,1.000000,folA


In [17]:
mgdata[(mgdata.lineage == 'F-G5-1')&(mgdata.position < 49764)&~(mgdata.ALT.isnull())]

,UID,position,lineage,REF,ALT,DP,AF,gene
842,6c16a376,7922,F-G5-1,CAA,CAAA,72.0,0.943662,yaaJ
1116,5b2c41ec,10647,F-G5-1,A,G,59.0,1.000000,yaaW
1316,e3ee0d63,14301,F-G5-1,C,T,61.0,1.000000,dnaJ
2885,72168f09,28269,F-G5-1,C,T,71.0,0.985915,NaN
2958,ee45d76d,30616,F-G5-1,T,C,60.0,1.000000,carA
3561,8c7c87b4,35373,F-G5-1,G,C,75.0,1.000000,caiE
3894,1985685c,37903,F-G5-1,C,T,56.0,0.982143,caiB
4964,0ad54f64,47776,F-G5-1,G,A,57.0,1.000000,kefC


# Check C321 Hitchhiker mutations

Go through the C321 alignment to MG1655 to put together a list of hitchiker mutations

In [18]:
hhdata = pd.read_csv("variants_C321g0toMG1655.csv",dtype = {'IS_HET': str, 'AO': str, 'QA': str} )
hhdata.sort_values(by='lineage')
hh_avg_readdepth = hhdata.DP.mean()

print("total C321 gen0 to MG1655 variant calls: ",hhdata.UID.count(),
      " -- at an average read depth of: ", round(hh_avg_readdepth,1))
print("variant calls under min read depth: ",hhdata[hhdata.DP < depth].UID.count())
print("variant MG1655 calls under min allele fraction: ",hhdata[hhdata.AF < fraction].UID.count())
print()

hhc = hhdata[((hhdata.DP >= depth) & (hhdata.REF.str.len() < long)) |
                 ((hhdata.DP >= depth_long) & (hhdata.REF.str.len() > long))]
hhc = hhc[((hhc.AF >= fraction) & (hhc.REF.str.len() < long)) | 
          ((hhc.AF >= fraction_long) & (hhc.REF.str.len() > long))]

print("total C321 gen0 to MG1655 culled variant calls: ",hhc.UID.count())
print()

hhc_noTAG = hhc[~hhc.position.isin(positions)]

print("C321 gen0 to MG1655 culled hitchiker calls, with TAG-TAA conversions removed: ",hhc_noTAG.UID.count())
print("apparent TAAs removed:",hhc.UID.count()-hhc_noTAG.UID.count())


KeyError: 'lineage'

Take list of hitchikers and begin analysis...

In [ ]:
mg_HHs = mgdata[(mgdata.position.isin(hhc_noTAG.position)&(mgdata.lineage.str[0]!='E'))].loc[:,['position','lineage','gene','REF','ALT','DP','AF']]
hh_group = mg_HHs.groupby(['position'])
hh_altgroup = mg_HHs.groupby(['position','ALT'])
samples = mg_HHs.lineage.nunique()
positions = mg_HHs.position.nunique()
variants = mg_HHs[~mg_HHs.ALT.isnull()].index.size
                 
underhit_hh_altgroup = hh_altgroup.filter(lambda x: len(x) < samples)
overhit_hh_altgroup = hh_altgroup.filter(lambda x: len(x) > samples)
underhit_hhpos = pd.DataFrame()
overhit_hhpos = pd.DataFrame()

for pos in underhit_hh_altgroup.position.unique():
    underhit_hhpos = pd.concat([underhit_hhpos,hh_group.get_group(pos)])
for pos in overhit_hh_altgroup.position.unique():
    overhit_hhpos = pd.concat([overhit_hhpos,hh_group.get_group(pos)])

underhit_hhs = underhit_hhpos[underhit_hhpos.ALT.isnull()&underhit_hhpos.DP.notnull()]
#filter out one C-G0 parental mutation, 13 F11 starter mutations, and extraneous junk:
underhit_hhs = underhit_hhs.groupby('position').filter(lambda x: len(x) < 3)

print("There are",positions,"hitchiker positions found, which times",samples,
      "samples aligned, should be",samples*positions,
      "lines.\nNumber of called variants is:",variants,", or",
      samples*positions-variants,"fewer than predicted.\n")
print("There are",underhit_hhs.position.nunique(),"positions hit fewer than ",samples,
      " times, which are: \n",underhit_hhs.position.unique())
print("The positions UIDs hit more than ",samples," times are: \n",overhit_hh_altgroup.position.unique())
print("Underhit sample positions:\n",underhit_hhs.gene.value_counts())

In [ ]:
mg_TAAs

In [ ]:
underhit_hhs.groupby('position').filter(lambda x: len(x) < 26).position.value_counts()

# Look through Dan Rice's Data

In [ ]:
drHom = pd.read_csv("samples_c321_homoplasies_vx.csv")
compare = pd.merge(left=drHom,right=c3variants, left_on='POS', right_on='position')
compare = compare[['POS','REF_x','REF_y','ALT_x','ALT_y','gene','Samples_with_mutation']].sort_values(by='gene').drop_duplicates()
compare.to_csv('drc3Compare.csv')

In [ ]:
pseudo = False
if not pseudo:
    print('not')

In [ ]:


mgdata.loc['letter1'] = [x[0] for x in mgdata.columns.tolist()]
mgdata.loc['letter2'] = [x[1] for x in mgdata.columns.tolist()]
mgdata = mgdata[mgdata.columns[mgdata.loc['letter2'].argsort()]]
mgdata = mgdata[mgdata.columns[mgdata.loc['letter1'].argsort()]]

In [ ]:
mgdata[]